In [1]:
%%capture
!pip install factor_analyzer
!pip install statsmodels

In [2]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("io.excel.xls.writer", "xlwt")
df = pd.read_pickle('~/Downloads/responses_latest.pkl')
df = df[
    (df['survey_short_id'] == 15) & (df['has_valid_response'] == True)
    ]

df[['scale_score', 'analysis_group', 'choice']] = df[['scale_score', 'analysis_group', 'choice']].fillna(0).astype(int)
df['question_text'] = df['question_text'].replace('<[^<]+?>', '', regex=True)

# A bit more cleanup
# Not doing audio for the moment
df = df[df.response_type != 'AU']
# Move CH and Scale Score to a new combined column
# https://stackoverflow.com/questions/29991862/copy-value-from-one-column-based-on-the-value-of-another-column
df['score'] = 0
df.loc[df['response_type'] == 'SC', 'score'] = df['scale_score']
df.loc[df['response_type'] == 'CH', 'score'] = df['choice']

As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.


/opt/anaconda3/lib/python3.8/site-packages/pandas/_config/config.py:630: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.
  warnings.warn(d.msg, FutureWarning)


In [3]:
#Create the pivot
df = df.pivot(index='respondent_id',
              columns=['analysis_group', 'question_sort_order', 'question_text', 'response_type'],
              values='score')
df.sort_index(axis=1, level=['analysis_group', 'question_sort_order'], inplace=True)
df.dropna(inplace=True)
df = df.astype('int16')
df

analysis_group                                                         0   \
question_sort_order                                                   10    
question_text       Zum Start, ein paar demografische Fragen.\r\nBist du:   
response_type                                                          CH   
respondent_id                                                               
865                                                                  1      
866                                                                  1      
887                                                                  2      
910                                                                  1      
913                                                                  1      
...                                                                ...      
1337                                                                 1      
1339                                                                 1      
1398                                                                 1      
1409                                                                 3      
1416                                                                 1      

analysis_group                                       \
question_sort_order         20                  30    
question_text       Dein Alter? Dein Familienstand?   
response_type                CH                  CH   
respondent_id                                         
865                           2                   4   
866                           5                   3   
887                           4                   2   
910                           4                   4   
913                           6                   4   
...                         ...                 ...   
1337                          4                   4   
1339                          4                   2   
1398                          3                   4   
1409                          5                   1   
1416                          4                   1   

analysis_group                                   \
question_sort_order                         40    
question_text       Deine berufliche Situation?   
response_type                                CH   
respondent_id                                     
865                                           1   
866                                           1   
887                                           3   
910                                           2   
913                                           1   
...                                         ...   
1337                                          3   
1339                                          2   
1398                                          1   
1409                                          1   
1416                                          2   

analysis_group                                                                               \
question_sort_order                                                                     44    
question_text       Nun geht es um das Thema Covid Impfung.\r\n\r\nWie ist Dein Impfstatus?   
response_type                                                                            CH   
respondent_id                                                                                 
865                                                                  1                        
866                                                                  1                        
887                                                                  3                        
910                                                                  2                        
913                                                                  1                        
...                                                                ...                        
1337                    

In [4]:
#https://www.datacamp.com/community/tutorials/introduction-factor-analysis
#do the factor analysis
# cols = list(range(1, 15))

#don't need any without the right group. In our case, groups are 1 - 14
df_4fa = df.copy(deep=True)
df_4fa = df_4fa.drop(0, axis=1, level=0)
df_4fa.columns = df_4fa.columns.to_flat_index()

df_4fa

,"(1, 80, Ich genieße es, mich in komplexe Sachverhalte einzuarbeiten., SC)","(1, 220, Es gefällt mir, Dinge zu untersuchen., SC)","(1, 360, Ich liebe es, zu denken., SC)","(2, 60, Ich lege großen Wert darauf, dass andere mich sympathisch finden., SC)","(2, 200, Ich fühle mich unwohl, wenn ich Dinge mache, die andere schlecht finden., SC)","(2, 340, Ich vermeide es, einen schlechten Eindruck auf andere zu machen., SC)","(3, 90, Ich fühle mich wohl damit, wenn ich Entscheidungen für andere treffen muss., SC)","(3, 230, Ich fühle mich wohl in der Rolle des Anführers., SC)","(3, 370, Es gefällt mir, wenn ich die Verantwortung für wichtige Entscheidungen trage., SC)","(4, 180, Meine Stellung in der Gesellschaft ist mir sehr wichtig., SC)","(4, 320, Ich suche nach Situationen, in denen ich mein soziales Ansehen erhöhen kann., SC)","(4, 460, Ich freue mich über die Stellung in der Gesellschaft, die ich erreicht habe., SC)","(5, 110, Es ist mir sehr wichtig, sparsam mit meinen Vorräten umzugehen., SC)","(5, 250, Ich bemühe mich, mein Geld und Gut zu erhalten., SC)","(5, 390, Andere würden mich als preisbewusst bezeichnen., SC)","(6, 150, Ich komme allein gut klar im Leben., SC)","(6, 290, Ich komme am besten klar im Leben, wenn ich mich nur auf mich selbst verlasse., SC)","(6, 430, Es wäre schrecklich für mich, von anderen abhängig zu sein., SC)","(7, 130, Meine Freundschaften und Bekanntschaften bereichern mein Leben., SC)","(7, 270, Ich lerne gerne neue Leute kennen., SC)","(7, 410, Ich genieße Gespräche und Plaudereien., SC)","(8, 170, Ich achte auf Sitten und Gebräuche, auf die Regeln des Zusammenlebens., SC)","(8, 310, Ich bin stolz auf meine moralischen Überzeugungen., SC)","(8, 450, Es fällt mir schwer, von Moralvorstellungen abzuweichen., SC)","(9, 100, Ich finde es wichtig, mich um Notleidende zu kümmern., SC)","(9, 240, Ich möchte einen Beitrag leisten, damit unsere Welt lebenswerter wird., SC)","(9, 380, Ich finde es furchtbar, Not und Benachteiligung zu sehen., SC)","(10, 120, Ich genieße es, wenn meine Wohnung aufgeräumt ist., SC)","(10, 260, Wenn ich Durcheinander sehe, möchte ich sofort Ordnung machen., SC)","(10, 400, Ich liebe es, Dinge zu ordnen und zu strukturieren., SC)","(11, 160, Ich umgehe Situationen, die andere für gefährlich halten., SC)","(11, 300, Ich lege großen Wert auf Sicherheit., SC)","(11, 440, Ich fühle mich sehr unwohl, wenn ich eine Situation nicht einschätzen kann., SC)","(12, 140, Wenn mich jemand beleidigt, muss ich es ihm heimzahlen., SC)","(12, 280, Ich lasse es mir nicht gefallen, wenn mich jemand provoziert., SC)","(12, 420, Ich finde es wichtig, anderen deutlich zu zeigen, dass sie mich verletzt haben., SC)","(13, 50, Ab hier geht's um Deine Motivationen. \r\n\r\nIch genieße es, Sport zu treiben., SC)","(13, 190, Ich fühle mich wohl, wenn ich regelmäßig körperlich aktiv sein kann., SC)","(13, 330, Ich habe ein ausgeprägtes Bewegungsbedürfnis., SC)","(14, 70, Ich fühle mich sehr wohl auf Familienfesten., SC)","(14, 210, Ich achte auf Möglichkeiten, meine Familie unterstützen zu können., SC)","(14, 350, Ich denke häufig darüber nach, ob es den Mitgliedern meiner Familie gut geht., SC)"
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
865,3,2,2,5,5,5,4,4,5,4,3,4,5,4,5,4,2,5,6,4,6,3,1,1,2,2,4,6,6,6,6,6,6,5,5,6,6,5,5,5,4,3
866,3,3,3,5,6,6,5,5,5,2,1,5,3,3,3,5,5,2,2,1,1,1,1,1,2,2,2,4,4,4,2,2,1,2,2,2,5,6,6,2,1,1
887,4,4,5,4,3,3,2,3,4,4,3,4,5,3,1,5,3,3,3,4,4,3,4,4,4,4,3,4,5,4,4,4,4,4,3,3,3,4,4,3,4,3
910,5,6,6,4,3,4,4,5,5,4,3,5,4,4,3,6,4,4,6,5,5,3,5,4,5,6,6,4,4,4,2,4,5,2,4,5,6,6,5,4,6,5
913,4,2,4,3,2,3,4,2,4,2,2,4,4,4,3,4,3,3,4,2,3,4,4,4,4,4,3,4,2,3,4,4,4,3,2,3,1,2,2,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,3,2,4,4,4,5,3,4,5,4,3,6,3,4,4,3,3,5,6,4,4,6,5,5,4,6,5,5,3,3,5,5,4,1,3,6,4,5,4,5,6,5
1339,5,5,5,1,2,5,4,3,3,2,2,3,4,5,4,5,6,6,4,6,3,5,6,5,4,5,5,4,4,4

In [5]:
#https://www.datacamp.com/community/tutorials/introduction-factor-analysis
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
chi_square_value,p_value=calculate_bartlett_sphericity(df_4fa)
# p value < 0.05 means we reject the null hypothesis that all variances re the same- this is what we want
#chi_square_value, p_value

In [6]:
from factor_analyzer.factor_analyzer import calculate_kmo
kmo_all,kmo_model=calculate_kmo(df_4fa)
#https://www.statisticshowto.com/kaiser-meyer-olkin/
# Should be above .5
#kmo_model

In [8]:
# Create factor analysis object and perform factor analysis

from factor_analyzer import FactorAnalyzer
fa = FactorAnalyzer()
fa.fit(df_4fa)
# Check Eigenvalues
ev, v = fa.get_eigenvalues()
v

array([ 7.65535122,  4.20719912,  3.12109717,  1.89689868,  1.61136345,
        1.44120565,  0.99207148,  0.8564905 ,  0.74366102,  0.70151428,
        0.45967184,  0.33187691,  0.29782613,  0.17962055,  0.15178452,
        0.04984571,  0.02749724, -0.03561582, -0.0820789 , -0.10974987,
       -0.14827818, -0.18544474, -0.23401322, -0.27131339, -0.30423788,
       -0.32943602, -0.34335513, -0.3652185 , -0.3960207 , -0.41421481,
       -0.42709133, -0.45421252, -0.46906646, -0.4898349 , -0.52573007,
       -0.55801237, -0.56435017, -0.60073151, -0.64099315, -0.67000785,
       -0.68862763, -0.73362997])

In [9]:
fa = FactorAnalyzer(7, rotation="varimax")
fa.fit(df_4fa)

FactorAnalyzer(n_factors=7, rotation='varimax', rotation_kwargs={})

In [10]:
loadings = pd.DataFrame(fa.loadings_)
loadings.insert(0,'questions',df_4fa.columns)
loadings.to_excel('~/Downloads/loadings.xls')

In [10]:
# Get variance of each factors
pd.DataFrame(fa.get_factor_variance())

,0,1,2,3,4,5,6,7,8,9,10
0,4.386811,3.702931,3.074381,2.676022,2.355105,2.334488,2.248807,1.925255,1.746085,1.432401,1.396146
1,0.104448,0.088165,0.073200,0.063715,0.056074,0.055583,0.053543,0.045839,0.041573,0.034105,0.033242
2,0.104448,0.192613,0.265812,0.329527,0.385601,0.441184,0.494727,0.540567,0.582140,0.616245,0.649486


From here we assume the LUXX factors are sufficient - this is index = analysis group

In [11]:
idx = df.columns
#print(df.columns.names)
new_index_list = []
for i in idx:
    i_list = list(i)
    new_row = i_list[:]

    if i_list[1] == 44: # isolate dependent variable
        new_row[0] = 30
    
    else:
        if i_list[0] == 0: 
            new_row[0] = 20 # move demographic questions to end

    new_index_list.append(new_row)

#print(new_index_list)
new_index_list = zip(*new_index_list) # transpose the matrix, this needs to be in the right shape
new_index = pd.MultiIndex.from_arrays(new_index_list)
new_index.names = df.columns.names
df.columns = new_index
df.sort_index(axis=1, inplace=True)
df

analysis_group                                                                1   \
question_sort_order                                                          80    
question_text       Ich genieße es, mich in komplexe Sachverhalte einzuarbeiten.   
response_type                                                                 SC   
respondent_id                                                                      
865                                                                  3             
866                                                                  3             
887                                                                  4             
910                                                                  5             
913                                                                  4             
...                                                                ...             
1337                                                                 3             
1339                                                                 5             
1398                                                                 6             
1409                                                                 4             
1416                                                                 5             

analysis_group                                             \
question_sort_order                                   220   
question_text       Es gefällt mir, Dinge zu untersuchen.   
response_type                                          SC   
respondent_id                                               
865                                                     2   
866                                                     3   
887                                                     4   
910                                                     6   
913                                                     2   
...                                                   ...   
1337                                                    2   
1339                                                    5   
1398                                                    4   
1409                                                    5   
1416                                                    2   

analysis_group                                \
question_sort_order                      360   
question_text       Ich liebe es, zu denken.   
response_type                             SC   
respondent_id                                  
865                                        2   
866                                        3   
887                                        5   
910                                        6   
913                                        4   
...                                      ...   
1337                                       4   
1339                                       5   
1398                                       4   
1409                                       2   
1416                                       4   

analysis_group                                                                     2   \
question_sort_order                                                               60    
question_text       Ich lege großen Wert darauf, dass andere mich sympathisch finden.   
response_type                                                                      SC   
respondent_id                                                                           
865                                                                  5                  
866                                                                  5                  
887                                                                  4                  
910                                                                  4                  
913                                                                  3                  
...                

In [54]:
Y_vals = df.loc[:, (slice(None), 44)] #.to_numpy().flatten()
X_vals = df.drop(44, axis=1, level = 1).drop(20,axis=1, level=0)

X_vals_grouped = X_vals.groupby(axis=1, level=0).mean().to_numpy()

#Here we take the Luxx groups (motives) and use these 15 instead of the 45 factors


Y_vals.columns=[['dependent']]
Y_vals = Y_vals.to_numpy().flatten()
X_vals_grouped
Y_vals

array([1, 1, 3, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 4, 1, 3, 4, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 5, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 4, 1, 1, 3, 1, 1, 3, 1, 1, 1],
      dtype=int16)

In [87]:
import statsmodels.api as sm
Y = Y_vals
X = sm.add_constant(X_vals_grouped)  # Stats models doesn't assume a constant, you need to add one
pd.DataFrame(Y)
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.0,2.333333,5.000000,4.333333,3.666667,4.666667,3.666667,5.333333,1.666667,2.666667,6.000000,6.000000,5.333333,5.333333,4.000000
1,1.0,3.000000,5.666667,5.000000,2.666667,3.000000,4.000000,1.333333,1.000000,2.000000,4.000000,1.666667,2.000000,5.666667,1.333333
2,1.0,4.333333,3.333333,3.000000,3.666667,3.000000,3.666667,3.666667,3.666667,3.666667,4.333333,4.000000,3.333333,3.666667,3.333333
3,1.0,5.666667,3.666667,4.666667,4.000000,3.666667,4.666667,5.333333,4.000000,5.666667,4.000000,3.666667,3.666667,5.666667,5.000000
4,1.0,3.333333,2.666667,3.333333,2.666667,3.666667,3.333333,3.000000,4.000000,3.666667,3.000000,4.000000,2.666667,1.666667,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1.0,3.000000,4.333333,4.000000,4.333333,3.666667,3.666667,4.666667,5.333333,5.000000,3.666667,4.666667,3.333333,4.333333,5.333333
60,1.0,5.000000,2.666667,3.333333,2.333333,4.333333,5.666667,4.333333,5.333333,4.666667,4.000000,3.000000,3.000000,3.666667,5.333333
61,1.0,4.666667,3.333333,5.000000,2.666667,3.333333,4.333333,6.000000,2.666667,5.000000,2.666667,1.666667,3.333333,5.333333,5.333333
62,1.0,3.666667,3.000000,4.000000,2.333333,2.666667,4.333333,5.333333,3.000000,5.333333,4.666667,4.000000,3.000000,5.000000,2.333333


In [114]:
#https://statisticsbyjim.com/regression/choosing-regression-analysis/
# OLS is wrong, need nominal logistic regression
# https://www.statsmodels.org/stable/discretemod.html
# WRONG model = sm.MNLogit(Y, X)
model = sm.MNLogit(Y, X) # does not converge - no solution
model_res = model.fit()

In [107]:
model_res.params

array([ 1.58973685,  0.12372678, -0.19080668, -0.42740214,  0.30424912,
       -0.26008165,  0.31180731,  0.38392868,  0.22928081, -0.29555714,
        0.06619095, -0.43539206,  0.20051029,  0.0566944 , -0.11860556])

In [100]:
model_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                      y   No. Observations:                   64
Model:                        MNLogit   Df Residuals:                        4
Method:                           MLE   Df Model:                           56
Date:                Sat, 18 Dec 2021   Pseudo R-squ.:                     nan
Time:                        21:46:08   Log-Likelihood:                    nan
converged:                       True   LL-Null:                       -53.007
Covariance Type:            nonrobust   LLR p-value:                       nan
==============================================================================
       y=2       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const             nan        nan        nan        nan         nan         nan
x1                nan        nan        nan        nan         nan         nan
x2                nan        nan        nan        nan         nan         nan
x3                nan        nan        nan        nan         nan         nan
x4                nan        nan        nan        nan         nan         nan
x5                nan        nan        nan        nan         nan         nan
x6                nan        nan        nan        nan         nan         nan
x7                nan        nan        nan        nan         nan         nan
x8                nan        nan        nan        nan         nan         nan
x9                nan        nan        nan        nan         nan         nan
x10               nan        nan        nan        nan         nan         nan
x11               nan        nan        nan        nan         nan         nan
x12               nan        nan        nan        nan         nan         nan
x13               nan        nan        nan        nan         nan         nan
x14               nan        nan        nan        nan         nan         nan
------------------------------------------------------------------------------
       y=3       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const             nan        nan        nan        nan         nan         nan
x1                nan        nan        nan        nan         nan         nan
x2                nan        nan        nan        nan         nan         nan
x3                nan        nan        nan        nan         nan         nan
x4                nan        nan        nan        nan         nan         nan
x5                nan        nan        nan        nan         nan         nan
x6                nan        nan        nan        nan         nan         nan
x7                nan        nan        nan        nan         nan         nan
x8                nan        nan        nan        nan         nan         nan
x9                nan        nan        nan        nan         nan         nan
x10               nan        nan        nan        nan         nan         nan
x11               nan        nan        nan        nan         nan         nan
x12               nan        nan        nan        nan         nan         nan
x13               nan        nan        nan        nan         nan         nan
x14               nan        nan        nan        nan         nan         nan
------------------------------------------------------------------------------
       y=4       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const             nan        nan        nan        nan         nan         nan
x1                nan        nan        nan        nan         nan         nan
x2